In [7]:
import os
os.chdir('/Users/sp/Desktop/CVIP/PA2Data')
import numpy as np
import cv2
from random import randint
img = cv2.imread('Butterfly.jpg')

In [8]:
"""Re-arrange the image into a feature vector matrix F containing both
spatial and range information - 333*500*3 image becomes 166500*5,
with two columns that represent pixel location"""
F = np.zeros([img.shape[0] * img.shape[1], 5], dtype=np.int)      

In [9]:
#Fill feature vector matrix
temp = np.zeros((1,3))
row = 0
for i in range(0,img.shape[0]):
    for j in range(0,img.shape[1]):      
        temp = img[i][j]       
        for k in range(0,5):
            if(k>=0) & (k <=2):
                F[row][k]=temp[k]
            else:
                if(k==3):
                    F[row][k]=i
                else:
                    F[row][k]=j
        row+=1
print ("Spatial range feature matrix created")


Spatial range feature matrix created


In [10]:
def euclid_distance(x, xi):
    return np.sqrt(np.sum((x - xi)**2))
def find_non_zero_indices(feature_matrix):
    indices=[]
    for index,value in enumerate(feature_matrix):       
        #.any() function can be thought of as logical or in python
        if value.any(): 
            indices.append(index)
    return indices

In [11]:
"""Set a single threshold h for this spatial-range feature matrix and
convergence criterion value iter"""
h = 90    #threshold
iter = 45 #convergence criterion value
iteration = 0
output = np.zeros([img.shape[0], img.shape[1], 3], np.uint8)


In [6]:
"""This process continues till you exhaust all the points"""
while(F.shape[0]!=0): 
    iteration += 1
    """Pick a random row from F as initial seed point which is current mean"""
    random_seed_index = np.random.randint(F.shape[0])
    seed = F[random_seed_index] #choose some random row as current mean
    output_matrix = np.zeros([F.shape[0], 5], dtype=np.int)
    marked_output_matrix = np.zeros([F.shape[0]]) 
    #add current mean to output image in place of the seed used to calculate mean
    output_matrix[random_seed_index] = seed  
    """compute the eucledian distance of all the points in F with current mean"""
    for each_point_index in range(F.shape[0]):
        each_point = F[each_point_index]
        distance = euclid_distance(seed,each_point)
        """store the indexes of all the points that are within threshold h"""
        if(distance < h): #if distance < threshold (h) then add it to output_matrix
            output_matrix[each_point_index] = each_point
            #assign 1 to all the pixels that fall within the threshold
            marked_output_matrix[each_point_index] = 1 
         
    output_matrix_filtered = output_matrix[find_non_zero_indices(output_matrix)] 
    """Average out each column of these select points and find the new mean"""
    output_matrix_mean = np.mean(output_matrix_filtered, axis=0)
    euc_distance = euclid_distance(seed,output_matrix_mean)
    """check if the mean shift value is below iter. If yes, mark all these
        pixels in the original image with the mean intensity value. If no,
            repeat this iteration all over again"""
    if(euc_distance < iter):
        for each_point in range(output_matrix_filtered.shape[0]):
            for i in range(3):
                x = output_matrix_filtered[each_point][3]
                y = output_matrix_filtered[each_point][4]
                output[x][y][i] = seed[i] 
        F = np.delete(F, np.nonzero(marked_output_matrix)[0], 0)
print('Segmentation Done!')
cv2.imwrite("segmented_image.jpg", output)


Segmentation Done!


True